In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

#from tensorflow.keras import Sequential
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
#from tensorflow.keras.layers import Dense, Conv2D, Flatten, GlobalAveragePooling2D, Dropout
#from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
from keras import Sequential
from keras.optimizers import Adam
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from keras.layers import MaxPooling2D,Dense, Conv2D, Flatten, GlobalAveragePooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [3]:
import cv2
from PIL import Image

In [4]:
df = pd.read_csv("E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset/train.csv")
df_test = pd.read_csv("E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset/test.csv")

In [5]:
df.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [6]:
df["Class"].value_counts()

Food                    2278
Attire                  1691
misc                    1271
Decorationandsignage     743
Name: Class, dtype: int64

In [ ]:
# True image size
IMG_WIDTH = 120
IMG_HEIGHT = 80

def preprocess_image(image, sigmaX=10):
    """
    The whole preprocessing pipeline:
    1. Read in image
    2. Apply masks
    3. Resize image to desired size
    4. Add Gaussian noise to increase Robustness
    
    :param img: A NumPy Array that will be cropped
    :param sigmaX: Value used for add GaussianBlur to the image
    
    :return: A NumPy array containing the preprocessed image
    """
    image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
    image = np.array(image)
    return image

In [ ]:
y = df["Class"]

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
#img = cv2.imread(r'E:/Data Science/Kaggle Competitions dataset/Hackerearth gala/dataset/Train Images/image7042.jpg')
#cv2.imshow('image',img)
#cv2.waitKey(0)

In [ ]:
# MAKING SURE ALL IMAGES ARE OF SAME SIZE

diff_shape = set()
PATH = "E:/Data Science/Kaggle Competitions dataset/Hackerearth gala/dataset/Train Images/"
for i in range(df.shape[0]):
    temp = PATH + str(df["Image"][i])
    img = cv2.imread(temp)
    diff_shape.add(img.shape[0])
    diff_shape.add(img.shape[1])
    diff_shape.add(img.shape[2])

In [ ]:
BATCH_SIZE = 16
# resizing the image to the size of efficientnet
#IMG_WIDTH = 456
#IMG_HEIGHT = 456

# Add Image augmentation to our generator
train_datagen = ImageDataGenerator(rotation_range=360,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split=0.2,
                                   preprocessing_function=preprocess_image, 
                                   rescale=1 / 128.)
train_generator = train_datagen.flow_from_dataframe(df, 
                                                    x_col='Image', 
                                                    y_col='Class',
                                                    directory = PATH,
                                                    target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical', 
                                                    subset='training')
val_generator = train_datagen.flow_from_dataframe(df, 
                                                  x_col='Image', 
                                                  y_col='Class',
                                                  directory = PATH,
                                                  target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='categorical',
                                                  subset='validation')

In [ ]:
## seems like efficient net is too large
# can try to freeze some layers


#from keras_efficientnets import EfficientNetB5
#effnet = EfficientNetB5(input_shape=(456,456,3),
#                        weights='imagenet',
#                        include_top=False)

In [ ]:
#effnet.summary()

In [ ]:
from keras.regularizers import l2
l2 = l2(0.01)

In [ ]:
def build_model():
    """
    A custom implementation of EfficientNetB5
    for the APTOS 2019 competition
    (Regression)
    """
    model = Sequential()
    
    model.add(Conv2D(filters = 100, kernel_size = (3,3),padding = 'Same',data_format="channels_last", strides = 1, 
                 activation ='relu', input_shape = (IMG_HEIGHT,IMG_WIDTH,3),kernel_initializer = "lecun_uniform",kernel_regularizer = l2))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

    
    model.add(Conv2D(filters = 200, kernel_size = (3,3),padding = 'Same', strides = 1, 
                 activation ='relu',kernel_initializer = "lecun_uniform",kernel_regularizer = l2))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

    
    model.add(Conv2D(filters = 100, kernel_size = (5,5),padding = 'same', strides = 1, 
                 activation ='relu',kernel_initializer = "lecun_uniform",kernel_regularizer = l2))
    model.add(Conv2D(filters = 100, kernel_size = (5,5),padding = 'same', strides = 1, 
                 activation ='relu',kernel_initializer = "lecun_uniform",kernel_regularizer = l2))
    model.add(Conv2D(filters = 100, kernel_size = (5,5),padding = 'same', strides = 1, 
                 activation ='relu',kernel_initializer = "lecun_uniform",kernel_regularizer = l2))
    model.add(Conv2D(filters = 100, kernel_size = (7,7),padding = 'same', strides = 1, 
                 activation ='relu',kernel_initializer = "lecun_uniform",kernel_regularizer = l2))
    model.add(Conv2D(filters = 100, kernel_size = (7,7),padding = 'same', strides = 1, 
                 activation ='relu',kernel_initializer = "lecun_uniform",kernel_regularizer = l2))
    

    model.add(Conv2D(filters = 100, kernel_size = (7,7),padding = 'same', strides = 1, 
                 activation ='relu',kernel_initializer = "lecun_uniform",kernel_regularizer = l2))


    model.add(Conv2D(filters = 100, kernel_size = (5,5),padding = 'same', strides = 1, 
                 activation ='relu',kernel_initializer = "lecun_uniform",kernel_regularizer = l2))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(20, activation="relu"))
    model.add(Dense(4, activation="softmax"))
    model.compile(loss='categorical_crossentropy',
                  optimizer="adam", 
                  metrics=['categorical_crossentropy', 'mse'])
    print(model.summary())
    return model

# Initialize model
model = build_model()

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=12)
rlr = ReduceLROnPlateau(monitor='val_loss', 
                        factor=0.5, 
                        patience=15,
                        verbose=1, 
                        mode='auto', 
                        epsilon=0.00001)
from keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint(r'E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset/my_architecture.h5', monitor='categorical_crossentropy', mode='min', verbose=1, save_best_only=True)
# Begin training
model.fit_generator(train_generator,
                    steps_per_epoch=train_generator.samples // BATCH_SIZE,
                    epochs=100,
                    validation_data=val_generator,
                    validation_steps = val_generator.samples // BATCH_SIZE,
                    callbacks=[mc, rlr])
